In [ ]:
!pip install boto3 numpy nibabel matplotlib tensorflow scikit-image


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
# Import required libraries
import boto3
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import io
import tempfile
import os

# Initialize S3 resource
s3 = boto3.resource('s3') # Creates an S3 resource to interact with Amazon S3
bucket_name = 'chemocraft-data' # Name of S3 bucket with data files
folder_path = 'Data/BraTS20_Training_369 copy/' # Folder path in bucket with the files 
#folder_path = 'MICCAI_BraTS2020_TrainingData/' # New Folder path in bucket with all the files 
save_path = 'Images'
bucket = s3.Bucket(bucket_name) # Gets specific S3 bucket needed 



CROP_MARGINS = {'left': 20, 'right': 10, 'top': 30, 'bottom': 30}

def crop_image(data_slice, crop_margins):
    top, bottom, left, right = crop_margins['top'], crop_margins['bottom'], crop_margins['left'], crop_margins['right']
    return data_slice[top:data_slice.shape[0] - bottom, left:data_slice.shape[1] - right]

# Define function to fetch, load and display the file from S3
def render_nii_from_s3(filename):
    print(f"Fetching file: {filename}") # Print message indicating the file being fetched

    # Retrieve the file from S3 and store it in a memory stream
    obj = bucket.Object(folder_path + filename) # Accesses the specific file in S3 bucket
    file_stream = io.BytesIO(obj.get()['Body'].read()) # Reads file content into an in-memory

    # Save the file to a temporary location 
    with tempfile.NamedTemporaryFile(suffix='.nii', delete=False) as temp_file:  # Disable auto-delete
        temp_file.write(file_stream.getvalue()) # Write the in-memory data to temporary file
        temp_file.flush() # Ensure all the data is written to disk

        temp_file_path = temp_file.name # Store the path of the temporary file
        print(f"Temporary file created: {temp_file_path}") # Confirm creation of temporary file

    # Try loading and displaying the data
    try:
        img = nib.load(temp_file_path) # Load the file into nibabel image object
        data = img.get_fdata() # Extract the image data as a numpy array

        print(f"Data shape for {filename}: {data.shape}") # Print shape of the data array for debugging

        # Check if the data array is empty
        if data.size == 0:
            print(f"No data found in {filename}") # Print message if the data array is empty
            return
        
        # Select the middle slice of the 3D data along the third dimension for visualization
        slice_idx = data.shape[2] // 2 # Calculate the index of the middle slice
        cropped_slice = crop_image(data[:,:,slice_idx], CROP_MARGINS)

        # Display the selected slice using matplotlib
        plt.figure(figsize=(6, 6)) # Create a new figure with specified size
        plt.imshow(cropped_slice, cmap='gray') # Display the middle slice in grayscale
        plt.title(f'{filename} - Slice {slice_idx}') # Add a title to indicate file and slice
        plt.axis('off')  # Hide axes for cleaner display
        plt.show() # Render the plot

    except Exception as e:
        print(f"Error loading file {filename}: {e}") # Print error message if any exception occurs 
    
    # Ensure temporary file is deleted to free up space
    finally:
        try:
            os.remove(temp_file_path)  # Clean up the temp file
            print(f"Deleted temporary file: {temp_file_path}") # Confirm file deletion
        except OSError as cleanup_error: # Handle any errors that may occur during deletion
            print(f"Error deleting temp file: {cleanup_error}") # Print error message if deletion fails

# def from_nii_to_png(filename):
def from_nii_to_png(key):
    # print(f"Fetching file: {filename}")
    print(f"Fetching file: {key}")

    # obj = bucket.Object(folder_path + filename) 
    obj = bucket.Object(key) 
    file_stream = io.BytesIO(obj.get()['Body'].read())

    # total_file_name = filename.split(".")[0]
    total_file_name = key.split(".")[0]
    brain_number = total_file_name.split("_")[-2]
    brain_scan_type = total_file_name.split("_")[-1]
    brain_slice_path = f"{save_path}/{brain_number}/{brain_scan_type}/"
    print(f"Saving in directory: {brain_slice_path}")

    with tempfile.NamedTemporaryFile(suffix='.nii', delete=False) as temp_file: 
        temp_file.write(file_stream.getvalue()) 
        temp_file.flush() 
        temp_file_path = temp_file.name 

        try:
            img = nib.load(temp_file_path) 
            data = img.get_fdata() 

            os.makedirs(brain_slice_path, exist_ok=True)
            
            if data.size == 0:
                print(f"No data found in {filename}")
                return
            
            for slice_idx in range(data.shape[2]):
                cropped_slice = crop_image(data[:,:,slice_idx], CROP_MARGINS)
                # slice_path = os.path.join(save_path, f'{slice_idx}.png') # "Images/Slice_Index"
                png_filename = f"{brain_slice_path}{slice_idx}.png"
                try:
                    # print(f"Saving: {png_filename}") 
                    mpimg.imsave(png_filename, cropped_slice, cmap='gray')
                except Exception as e:
                    print(f"Error saving file: {e}")
                
                try:
                    print(f"uploading {png_filename} to S3")
                    s3.Bucket(bucket_name).upload_file(png_filename, f"Ayomide/{png_filename}")
                    # print(f"deleteing {png_filename}")
                    os.remove(png_filename)
                except Exception as e:
                    print(f"Error: Could not upload PNG file: {e}")
        except Exception as e:
            (f"Error saving file {filename}: {e}") 

        

found_files = False # Flag to check if any .nii files are found
for obj in bucket.objects.filter(Prefix=folder_path): # Iterate over objects in specified folder 
    print(f"Found object key: {obj.key}")
    if obj.key.endswith('.nii'): # Process files with .nii extension only 
        found_files = True # Set flag to True if a .nii file is found
        filename = obj.key.split('/')[-1]  # Get the filename
        #from_nii_to_png(filename)
        from_nii_to_png(obj.key)
        # render_nii_from_s3(filename) # Call function to fetch, load and display files

if not found_files:
    print(f"No .nii files found in the folder {folder_path}") # Print message if no .nii files were found

Found object key: Data/BraTS20_Training_369 copy/BraTS20_Training_369_flair.nii
Fetching file: Data/BraTS20_Training_369 copy/BraTS20_Training_369_flair.nii
Saving in directory: Images/369/flair/
uploading Images/369/flair/0.png to S3
uploading Images/369/flair/1.png to S3
uploading Images/369/flair/2.png to S3
uploading Images/369/flair/3.png to S3
uploading Images/369/flair/4.png to S3
uploading Images/369/flair/5.png to S3
uploading Images/369/flair/6.png to S3
uploading Images/369/flair/7.png to S3
uploading Images/369/flair/8.png to S3
uploading Images/369/flair/9.png to S3
uploading Images/369/flair/10.png to S3
uploading Images/369/flair/11.png to S3
uploading Images/369/flair/12.png to S3
uploading Images/369/flair/13.png to S3
uploading Images/369/flair/14.png to S3
uploading Images/369/flair/15.png to S3
uploading Images/369/flair/16.png to S3
uploading Images/369/flair/17.png to S3
uploading Images/369/flair/18.png to S3
uploading Images/369/flair/19.png to S3
uploading Imag

In [32]:
import boto3
import numpy as np
import io
from PIL import Image

bucket_name = 'chemocraft-data'
folder_path = 'Ayomide/Images/'

def load_images_from_s3(bucket_name, folder_path):
    """Loads grayscale images from an S3 folder, normalizes, and returns as a NumPy array."""
    s3 = boto3.client('s3')
    images = []

    # Attempt to list objects in the specified S3 folder
    try:
        response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_path)
        
        # Check if 'Contents' key exists in response
        if 'Contents' not in response:
            print(f"No contents found in folder {folder_path}")
            return np.array([])  # Return empty array if folder is empty
        
        # Iterate over objects and load .png images
        for obj in response['Contents']:
            if obj['Key'].endswith('.png'):
                file_obj = s3.get_object(Bucket=bucket_name, Key=obj['Key'])
                file_content = file_obj['Body'].read()
                
                # Convert image to grayscale and normalize to [-1, 1]
                image = Image.open(io.BytesIO(file_content)).convert('L')
                image_array = np.array(image) / 127.5 - 1.0
                images.append(image_array)
        
        #print(f"Loaded {len(images)} images from {folder_path}")
        return np.array(images)  # Convert list of images to NumPy array
        
    except Exception as e:
        print(f"Error loading images from S3: {e}")
        return np.array([])  # Return empty array on error
    
images = load_images_from_s3(bucket_name, folder_path)

In [33]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from skimage.metrics import structural_similarity as ssim
from tensorflow.keras.layers import UpSampling2D


def build_generator(latent_dim):
       model = models.Sequential()
       model.add(layers.Dense(128 * 64 * 64, activation="relu", input_dim=latent_dim))
       model.add(layers.Reshape((64, 64, 128)))
       model.add(layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same", activation="relu"))
       model.add(layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same", activation="relu"))
       model.add(layers.Conv2D(1, kernel_size=3, strides=1, padding="same", activation="tanh"))
       return model

# Discriminator
def build_discriminator(img_shape):
    # Initialize sequential model
    model = models.Sequential()
    # First convolution layer: extracts features, downsampling to (img_shape[0]/2, img_shape[1]/2)
    model.add(layers.Conv2D(64, kernel_size=4, strides=2, padding="same", input_shape=img_shape, activation="relu"))
    # Second convolution layer: further extracts features, downsampling to (img_shape[0]/4, img_shape[1]/4)
    model.add(layers.Conv2D(128, kernel_size=4, strides=2, padding="same", activation="relu"))
    # Flatten the feature maps into a 1D vector
    model.add(layers.Flatten())
    # Output layer: predicts whether the image is real or fake
    model.add(layers.Dense(1, activation="sigmoid"))
    # Return the discriminator model
    return model

# Compile GAN
def compile_gan(generator, discriminator, latent_dim):
    discriminator.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    z = layers.Input(shape=(latent_dim,))
    img = generator(z)
    discriminator.trainable = False
    validity = discriminator(img)
    gan = models.Model(z, validity)
    gan.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return gan

# Training Function
def train_gan(generator, discriminator, gan, train_slices, latent_dim, epochs, batch_size):
    for epoch in range(epochs):
        for _ in range(len(train_slices) // batch_size):
            # Select random batch of real images
            idx = np.random.randint(0, len(train_slices), batch_size)
            real_slices = np.array([train_slices[i] for i in idx])

            # Generate fake images
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            fake_slices = generator.predict(noise)

            # Train the discriminator
            d_loss_real = discriminator.train_on_batch(real_slices, np.ones((batch_size, 1)))
            d_loss_fake = discriminator.train_on_batch(fake_slices, np.zeros((batch_size, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train the generator
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

        print(f"Epoch {epoch + 1}/{epochs}, D Loss: {d_loss[0]}, G Loss: {g_loss[0]}")

# Generate 3D Image
def generate_3d_image(generator, latent_dim, num_slices=150):
    noise = np.random.normal(0, 1, (num_slices, latent_dim))
    generated_slices = generator.predict(noise)
    generated_3d_image = np.stack(generated_slices, axis=0)  # Stack along the first dimension
    return generated_3d_image

# Evaluate SSIM for 3D Images
def evaluate_3d_image(true_image, generated_image):
    ssim_scores = []
    for i in range(true_image.shape[0]):  # For each slice
        slice_ssim = ssim(true_image[i], generated_image[i], multichannel=True)
        ssim_scores.append(slice_ssim)
        print(f"SSIM for slice {i}: {slice_ssim}")
    average_ssim = np.mean(ssim_scores)
    print(f"Average SSIM for entire 3D image: {average_ssim}")


In [ ]:
latent_dim = 100  # Latent space dimensionality
img_shape = (64, 64, 1)  # Shape of the images (64x64 with 1 channel)

generator = build_generator(latent_dim)
discriminator = build_discriminator(img_shape)
gan = compile_gan(generator, discriminator, latent_dim)

epochs = 10000
batch_size = 32

train_gan(generator, discriminator, gan, train_slices, latent_dim, epochs, batch_size)

num_slices = 150  # Number of slices in the 3D image
generated_3d_image = generate_3d_image(generator, latent_dim, num_slices)

evaluate_3d_image(true_image, generated_3d_image)  # If you have the true_image to compare against